In [1]:
import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
# from transformers import BertForSequenceClassification, BertConfig, BertModel
from transformers import LongformerModel, LongformerTokenizer

import pandas as pd
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.utils import class_weight
import numpy as np
from torch.optim import Adam
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
import json
import random

In [2]:
pd.options.display.max_colwidth = 300
pd.options.display.max_rows = 300

In [3]:
device = torch.device('cpu')
device

device(type='cpu')

In [4]:
data=pd.read_csv('./master_data_V3_0.csv')

/tmp/ipykernel_26005/4152142652.py:1: DtypeWarning: Columns (6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('./master_data_V3_0.csv')


In [5]:
data.columns

Index(['ADR', 'text', 'category', 'type', 'date', 'ogr/aug', 'page#', 'angle',
       'renamed category'],
      dtype='object')

In [6]:
# data[data['type']=='NMIC']['ADR'].unique()

In [7]:
# #data = pd.read_csv('./Backup master data/master_data_6.csv', usecols=['ADR', 'text', 'category', 'type', 'date'])
# old_data = pd.read_csv('./oldnew.csv')
# data = pd.read_csv('./master_data_new.csv')
# print(old_data.shape)
# print(data.shape)


In [8]:
data['category'].value_counts()

Tax Return - Personal                                 132733
Bank Statement                                         43085
Hazard Insurance Dec Page - Initial                    23231
Tax Return - Business                                  20919
Retirement Account Statement(s)                        12716
Paystub                                                12358
Purchase Agreement                                     10055
W-2(s) / 1099(s)                                        9039
Mortgage Statement                                      7744
Condominium Documents                                   6348
Rental Agreements(s)                                    5778
Driver's License                                        5633
Divorce Decree                                          4683
Divorce Decree / Child Support                          3619
Title Report                                            3175
Escrow/Closing Instructions                             1995
K1s                     

In [9]:
# old_data_cat=old_data['category'].unique()

In [10]:
# new_data_cat=data['category'].unique()

In [11]:
# not_in_newdata=set()
# for cat in old_data_cat:
#     if cat not in new_data_cat:
#         not_in_newdata.add(cat)

In [12]:
# old_data_cat

In [13]:
# for dd in not_in_newdata:
#     print(dd)
# #     print(old_data[old_data['category']==dd]['org/aug'].value_counts())
#     tdf=pd.DataFrame(old_data[(old_data['category']==dd) & (old_data['org/aug']=='org')])
#     print(tdf.shape[0])
#     tdf['page#']=[-1]*tdf.shape[0]
#     tdf['angle']=[0]*tdf.shape[0]
# #     break
#     data=data.append(tdf)

In [14]:
data.isnull().sum()

ADR                     0
text                 1649
category                0
type                    0
date                    0
ogr/aug                 0
page#                   0
angle                  17
renamed category    84347
dtype: int64

In [15]:
data.dropna(inplace=True)
data.shape

(229177, 9)

In [16]:
data['category'].value_counts()

Tax Return - Personal                                 98667
Bank Statement                                        31728
Tax Return - Business                                 20913
Retirement Account Statement(s)                        9797
Paystub                                                8463
Hazard Insurance Dec Page - Initial                    8187
W-2(s) / 1099(s)                                       6617
Condominium Documents                                  6348
Rental Agreements(s)                                   5778
Mortgage Statement                                     5735
Purchase Agreement                                     4763
Divorce Decree / Child Support                         3619
Divorce Decree                                         3220
Title Report                                           3175
Escrow/Closing Instructions                            1995
K1s                                                    1389
Driver's License                        

In [17]:
# n = data[data['category']=='NotClassified'].index.tolist()
# data.drop(index= random.sample(n, 7), inplace=True)

In [18]:
temp = []
for category in data['category']:
    if category == 'Divorce Decree':
        temp.append('Divorce Decree / Child Support')
    elif category == 'Escrow / Closing Instructions':
        temp.append('Escrow/Closing Instructions')
    else:
#         temp.append('focused')
        temp.append(category)
data['category'] = temp

In [19]:
# n = data[data['category']=='Purchase Agreement'].index.tolist()
# data.drop(index= random.sample(n, 10000), inplace=True)

In [20]:
data['category'].value_counts()

Tax Return - Personal                                 98667
Bank Statement                                        31728
Tax Return - Business                                 20913
Retirement Account Statement(s)                        9797
Paystub                                                8463
Hazard Insurance Dec Page - Initial                    8187
Divorce Decree / Child Support                         6839
W-2(s) / 1099(s)                                       6617
Condominium Documents                                  6348
Rental Agreements(s)                                   5778
Mortgage Statement                                     5735
Purchase Agreement                                     4763
Title Report                                           3175
Escrow/Closing Instructions                            2384
K1s                                                    1389
Driver's License                                       1218
Letter of Explanation                   

## Training

In [21]:
temp = []

cats = ['Retirement Account Statement(s)',
        'Mortgage Statement',
        'Non-Mortgage Account Statement',
        'Bank Statement',
        'Purchase Agreement',
        'Rental Agreements(s)',
        'Hazard Insurance Dec Page - Initial',
        'Divorce Decree / Child Support',
        'Bankruptcy Papers']

# cats = ['Retirement Account Statement(s)',
#         'Mortgage Statement',
#         'Bank Statement']
        
        
for category in data['category']:
    if category not in cats:
        temp.append('others')
    else:
#         temp.append('focused')
        temp.append(category)
data['new_category'] = temp

In [22]:
data['new_category'].unique()

array(['others', 'Bank Statement', 'Retirement Account Statement(s)',
       'Divorce Decree / Child Support',
       'Hazard Insurance Dec Page - Initial', 'Mortgage Statement',
       'Purchase Agreement', 'Rental Agreements(s)', 'Bankruptcy Papers',
       'Non-Mortgage Account Statement'], dtype=object)

In [23]:
# label_dict = {}
# label_dict['focused'] = 1
# label_dict['others'] = 0
# label_dict

In [24]:
data.shape

(229177, 10)

In [25]:
possible_labels = data.new_category.unique()

label_dict = {}
label_dict['others']=0
for i,cat in enumerate(cats):
    label_dict[cat]=i+1
label_dict

{'others': 0,
 'Retirement Account Statement(s)': 1,
 'Mortgage Statement': 2,
 'Non-Mortgage Account Statement': 3,
 'Bank Statement': 4,
 'Purchase Agreement': 5,
 'Rental Agreements(s)': 6,
 'Hazard Insurance Dec Page - Initial': 7,
 'Divorce Decree / Child Support': 8,
 'Bankruptcy Papers': 9}

In [26]:
data['label'] = data.new_category.replace(label_dict)
data.head(1)

,ADR,text,category,type,date,ogr/aug,page#,angle,renamed category,new_category,label
0,ADR-2022-100004473184,"loanDepot Branch: 102 - Cerritos Branch - Cerritos, CA NMLS # 174457 Set-Up Submission Form Borrower: Suzi Builder Loan Officer: POS Banker Credit Reference: 116287576930000 DU Case file ID: 1626158786 C.O.E.: 8/7/2022 Submitted: 7/7/2022 Prospect Loan#: 3001618980 Sales Price: $500,500.00 Appr...",Setup Submission Form,LD,20220722123724,org,1,0.0,No,others,0


In [27]:
data = data.sample(frac=0.50)

In [28]:
df = data.sample(frac=0.80, random_state=10)
df.shape

(91670, 11)

In [29]:
test = data.drop(df.index).reset_index(drop=True)
test.shape

(22918, 11)

In [30]:
test['new_category'].value_counts()

others                                 15544
Bank Statement                          3114
Retirement Account Statement(s)          959
Hazard Insurance Dec Page - Initial      819
Divorce Decree / Child Support           706
Rental Agreements(s)                     593
Mortgage Statement                       560
Purchase Agreement                       493
Bankruptcy Papers                        105
Non-Mortgage Account Statement            25
Name: new_category, dtype: int64

In [31]:
df['new_category'].value_counts()

others                                 62018
Bank Statement                         12674
Retirement Account Statement(s)         3897
Hazard Insurance Dec Page - Initial     3335
Divorce Decree / Child Support          2729
Rental Agreements(s)                    2347
Mortgage Statement                      2295
Purchase Agreement                      1881
Bankruptcy Papers                        410
Non-Mortgage Account Statement            84
Name: new_category, dtype: int64

In [32]:
classweight = torch.tensor(class_weight.compute_class_weight(class_weight='balanced',
                                                             classes=df['new_category'].unique().tolist(),
                                                             y=df['new_category'].values.tolist()), dtype=torch.float)
classweight

tensor([  0.1478,   3.9058,   0.7233,   2.3523,   3.9943,   4.8735,   2.7487,
          3.3591,  22.3585, 109.1310])

In [33]:
df.reset_index(drop=True, inplace=True)
df.head(1)

,ADR,text,category,type,date,ogr/aug,page#,angle,renamed category,new_category,label
0,ADR-2022-100004560075,"Schedule 2 (Form 1040) 2021 SINGLETARY 241-25-0404 Page 2 Part ll Other Taxes (continued) 17 Other additional taxes: a Recapture of other credits. List type, form number, and amount 17a b Recapture of federal mortgage subsidy. If you sold your home in 2021, see instructions 17b c Additional tax...",Tax Return - Personal,LD,20220729062912,org,17,0.0,No,others,0


In [34]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.20, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

In [35]:
df['data_type'] = ['not_set']*df.shape[0]
df.head(2)

,ADR,text,category,type,date,ogr/aug,page#,angle,renamed category,new_category,label,data_type
0,ADR-2022-100004560075,"Schedule 2 (Form 1040) 2021 SINGLETARY 241-25-0404 Page 2 Part ll Other Taxes (continued) 17 Other additional taxes: a Recapture of other credits. List type, form number, and amount 17a b Recapture of federal mortgage subsidy. If you sold your home in 2021, see instructions 17b c Additional tax...",Tax Return - Personal,LD,20220729062912,org,17,0.0,No,others,0,not_set
1,ADR-2022-100004525039,"Schedule D Tax Worksheet 2019 Form 1040 Line 12a G Keep for your records Name(s) Shown on Return Social Security Number Ashley m Lelle 536-29-8223 1 a Enter your taxable income from Form 1040, line 11b 1 a 17,126. b Enter the amount from your (and your spouse’s) Form 2555, lines 45 and 50 b c A...",Tax Return - Personal,LD,20220727051802,org,42,0.0,No,others,0,not_set


In [36]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [37]:
df.groupby(['new_category', 'label', 'data_type']).count()

ADR   text  category  \
new_category                        label data_type                           
Bank Statement                      4     train      10139  10139     10139   
                                          val         2535   2535      2535   
Bankruptcy Papers                   9     train        328    328       328   
                                          val           82     82        82   
Divorce Decree / Child Support      8     train       2183   2183      2183   
                                          val          546    546       546   
Hazard Insurance Dec Page - Initial 7     train       2668   2668      2668   
                                          val          667    667       667   
Mortgage Statement                  2     train       1836   1836      1836   
                                          val          459    459       459   
Non-Mortgage Account Statement      3     train         67     67        67   
                                          val           17     17        17   
Purchase Agreement                  5     train       1505   1505      1505   
                                          val          376    376       376   
Rental Agreements(s)                6     train       1878   1878      1878   
                                          val          469    469       469   
Retirement Account Statement(s)     1     train       3118   3118      3118   
                                          val          779    779       779   
others                              0     train      49614  49614     49614   
                                          val        12404  12404     12404   

                                                      type   date  ogr/aug  \
new_category                        label data_type                          
Bank Statement                      4     train      10139  10139    10139   
                                          val         2535   2535     2535   
Bankruptcy Papers                   9     train        328    328      328   
                                          val           82     82       82   
Divorce Decree / Child Support      8     train       2183   2183     2183   
                                          val          546    546      546   
Hazard Insurance Dec Page - Initial 7     train       2668   2668     2668   
                                          val          667    667      667   
Mortgage Statement                  2     train       1836   1836     1836   
                                          val          459    459      459   
Non-Mortgage Account Statement      3     train         67     67       67   
                                          val           17     17       17   
Purchase Agreement                  5     train       1505   1505     1505   
                                          val          376    376      376   
Rental Agreements(s)                6     train       1878   1878     1878   
                                          val          469    469      469   
Retirement Account Statement(s)     1     train       3118   3118     3118   
                                          val          779    779      779   
others                              0     train      49614  49614    49614   
                                          val        12404  12404    12404   

                                                     page#  angle  \
new_category                        label data_type                 
Bank Statement                      4     train      10139  10139   
                                          val         2535   2535   
Bankruptcy Papers                   9     train        328    328   
                                          val           82     82   
Divorce Decree / Child Support      8     train       2183   2183   
                                          val          546    546   
Hazard Insurance Dec Page - Initial 7     train

In [38]:
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

In [39]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=2048, 
    truncation=True,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=2048,
    truncation=True,
    return_tensors='pt'
)

/data/mmortgage/MultiPage_Classification/classification_service/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2285: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [40]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

In [41]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [42]:
batch_size=1
dataloader_train = DataLoader(dataset_train, 
                              shuffle=True,
                              num_workers=0, 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   shuffle=False,
                                   num_workers=0, 
                                   batch_size=batch_size)

In [43]:
len(dataloader_train)

73336

In [44]:
### Longformer model commented - due to indexing issue -NP (trying bert model with longformer tokenizer)
# from torch import nn
# from transformers import LongformerModel,LongformerConfig

# class LongformerClassifier(nn.Module):

#     def __init__(self, 
#                  dropout=0.1, 
#                  LFconfig=False, 
#                  attention_heads=12, 
#                  hidden_layers=12, 
#                  numclass=13,
#                  hidden_dropout_prob=0.1,
#                  hidden_act='gelu',
#                  position_embedding_type='absolute'):
        
#         #self.attention_heads = attention_heads
#         super(LongformerClassifier, self).__init__()
        
#         if LFconfig:
#             configuration = LongformerConfig(num_attention_heads= attention_heads, 
#                                        num_hidden_layers= hidden_layers)   
#         else:
#             configuration = LongformerConfig()
#         #self.bert = BertModel.from_pretrained('bert-base-uncased', output_attentions=attention)
#         #print(configuration)
#         self.longformer = LongformerModel(configuration)
#         self.dropout = nn.Dropout(dropout)
#         self.linear = nn.Linear(768, numclass, bias=True)

#     def forward(self, input_id, mask):

#         _, pooled_output = self.longformer(input_ids= input_id, attention_mask=mask,return_dict=False)
#         dropout_output = self.dropout(pooled_output)
#         linear_output = self.linear(dropout_output)

#         return linear_output

In [45]:
from torch import nn
from transformers import LongformerModel,LongformerConfig

class BertClassifier(nn.Module):

    def __init__(self, 
                 dropout=0.1, 
                 BERTconfig=False, 
                 attention_heads=12, 
                 hidden_layers=12, 
                 numclass=1,
                 hidden_dropout_prob=0.1,
                 hidden_act='gelu',
                 position_embedding_type='absolute'):
        
        #self.attention_heads = attention_heads
        super(BertClassifier, self).__init__()
        
        if BERTconfig:
            configuration = LongformerConfig(num_attention_heads= attention_heads, 
                                       num_hidden_layers= hidden_layers)   
        else:
            configuration = LongformerConfig()
        self.bert = LongformerModel.from_pretrained('allenai/longformer-base-4096',
                                                  gradient_checkpointing=False,
                                                  attention_window = 512,return_dict=True)
        print(configuration)
#         self.bert = LongformerModel(configuration)
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, numclass, bias=True)

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)

        return linear_output

In [46]:
label_dict

{'others': 0,
 'Retirement Account Statement(s)': 1,
 'Mortgage Statement': 2,
 'Non-Mortgage Account Statement': 3,
 'Bank Statement': 4,
 'Purchase Agreement': 5,
 'Rental Agreements(s)': 6,
 'Hazard Insurance Dec Page - Initial': 7,
 'Divorce Decree / Child Support': 8,
 'Bankruptcy Papers': 9}

In [47]:
ATTENTION = 12
HIDDEN_LAYER = 1
CLASSES = len(label_dict)


model = BertClassifier(dropout=0.3, 
                       BERTconfig=True, 
                       attention_heads=ATTENTION,
                       hidden_layers=HIDDEN_LAYER,
                       numclass= CLASSES,
                       hidden_dropout_prob=0.3,
                       position_embedding_type='absolute')

model.to(device)
model.train()

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


LongformerConfig {
  "attention_probs_dropout_prob": 0.1,
  "attention_window": 512,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "longformer",
  "num_attention_heads": 12,
  "num_hidden_layers": 1,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "sep_token_id": 2,
  "transformers_version": "4.19.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



BertClassifier(
  (bert): LongformerModel(
    (embeddings): LongformerEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(4098, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): LongformerEncoder(
      (layer): ModuleList(
        (0): LongformerLayer(
          (attention): LongformerAttention(
            (self): LongformerSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (query_global): Linear(in_features=768, out_features=768, bias=True)
              (key_global): Linear(in_features=768, out_features=768, bias=True)
              (value_global): Linear(in_featu

In [48]:
# ATTENTION = 12
# HIDDEN_LAYER = 1
# CLASSES = len(label_dict)


# model = LongformerModel.from_pretrained('allenai/longformer-base-4096',
#                                                   gradient_checkpointing=False,
#                                                   attention_window = 512,return_dict=True)

# model.to(device)
# model.train()

In [49]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [50]:
# model = LongformerModel.from_pretrained('allenai/longformer-base-4096',
#                                                   gradient_checkpointing=False,
#                                                   attention_window = 512,
#                                                   num_labels = 10,
#                                                                      return_dict=True)

In [ ]:
EPOCH = 2

save_model = 250

criterion = nn.CrossEntropyLoss(weight=classweight.to(device), reduction='mean')

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8,
                  no_deprecation_warning=True)
                  

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=250,
                                            num_training_steps=len(dataloader_train)*EPOCH)


logs = []
i=0

for epoch in range(EPOCH):
    
    print('epoch--- ',epoch+1)
    
    for train_input in tqdm(dataloader_train):
        i=i+1
        model.zero_grad()
        input_id = train_input[0].to(device)
        mask = train_input[1].to(device)
        train_label = train_input[2].to(device)
        output = model(input_id, mask)
        batch_loss = criterion(output, train_label)
        batch_loss.backward()
        optimizer.step()
        scheduler.step()
        
        if(i%save_model==0):
            label_train =  torch.tensor([], dtype=torch.uint8).to(device)
            pred_train = torch.tensor([], dtype=torch.uint8).to(device)
            total_loss_train = 0

            for train_eval in dataloader_train:
                input_id = train_eval[0].to(device)
                mask = train_eval[1].to(device)
                train_label = train_eval[2].to(device)
                output = model(input_id, mask)
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                label_train = torch.cat((label_train, train_label), 0)
                pred_train = torch.cat((pred_train, output.argmax(dim=1)), 0)

            label_train = label_train.detach().cpu().numpy()
            pred_train = pred_train.detach().cpu().numpy()
            total_acc_train = accuracy_score(label_train, pred_train)
            total_f1_train = f1_score(label_train, pred_train, average='weighted')
            total_loss_train = total_loss_train/(len(dataloader_train)*batch_size)
            


            label_val =  torch.tensor([], dtype=torch.uint8).to(device)
            pred_val = torch.tensor([], dtype=torch.uint8).to(device)
            total_loss_val = 0

            for val_input in dataloader_validation:
                input_id = val_input[0].to(device)
                mask = val_input[1].to(device)
                val_label = val_input[2].to(device)
                output = model(input_id, mask)
                batch_loss = criterion(output, val_label)
                total_loss_val += batch_loss.item()
                label_val = torch.cat((label_val, val_label), 0)
                pred_val = torch.cat((pred_val, output.argmax(dim=1)), 0)

            label_val = label_val.detach().cpu().numpy()
            pred_val = pred_val.detach().cpu().numpy()
            total_acc_val = accuracy_score(label_val, pred_val)
            total_f1_val = f1_score(label_val, pred_val, average='weighted')
            total_loss_val = total_loss_val/(len(dataloader_validation)*batch_size)
            
            print('train_accuracy: ', total_acc_train, ', train_f1: ', total_f1_train, ', train_loss: ', total_loss_train)
            print('val_accuracy: ', total_acc_val, ',  val_f1: ', total_f1_val, ', val_loss: ', total_loss_val)

            torch.save(model.state_dict(), 
                       f'/mnt/custommodel/LF_encoder{HIDDEN_LAYER}_attention{ATTENTION}_epoch{i}.model')

            logs.append({'epoch': i,
                     'train_accuracy': total_acc_train,
                     'train_f1': total_f1_train,
                     'train_loss': total_loss_train,
                     'val_accuracy': total_acc_val,
                     'val_f1': total_f1_val,
                     'val_loss': total_loss_val})

epoch---  1


  0%|          | 249/73336 [44:49<218:55:41, 10.78s/it]

In [ ]:
import torch
from transformers import LongformerModel, LongformerTokenizer

model = LongformerModel.from_pretrained("allenai/longformer-base-4096")
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

SAMPLE_TEXT = " ".join(["Hello world! "] * 1000)  # long input document
input_ids = torch.tensor(tokenizer.encode(SAMPLE_TEXT)).unsqueeze(0)  # batch of size 1

attention_mask = torch.ones(
    input_ids.shape, dtype=torch.long, device=input_ids.device
)  # initialize to local attention
global_attention_mask = torch.zeros(
    input_ids.shape, dtype=torch.long, device=input_ids.device
)  # initialize to global attention to be deactivated for all tokens
global_attention_mask[
    :,
    [
        1,
        4,
        21,
    ],
] = 1  # Set global attention to random tokens for the sake of this example
# Usually, set global attention based on the task. For example,
# classification: the <s> token
# QA: question tokens
# LM: potentially on the beginning of sentences and paragraphs
outputs = model(input_ids, attention_mask=attention_mask, global_attention_mask=global_attention_mask)
sequence_output = outputs.last_hidden_state
pooled_output = outputs.pooler_output

In [ ]:
input_ids.shape